# Data Wrangling

### 0.0 Data Collection
    Goal: Organize data to streamline the next steps of your project.
        -Data loading  
        -Data joining

In [1]:
# Import libraries
#
# General
#
import datetime as dt
import glob
import os

#
# Visualization
#
import matplotlib.pyplot as plt
import numpy as np
#
# Math and computation
#
import pandas as pd
import seaborn as sns

#
# Modeling
#

#### 0.1 Data Loading
    In this competition data are found in the jpx-tokyo-stock-exchange-prediction folder. This data is separated into training, supplemental, data 
    specification, and example test files. 

In [2]:
# Here we define the paths used to retrieve the data

# Directory path (laptop)
dir_path = '/Users/t/Desktop/Notebook/SpringBoard/CapstoneTwo/jpx-tokyo-stock-exchange-prediction.nosync'

# Let's read in the data separately first to evaluate it
stock_prices_path = dir_path + '/train_files/stock_prices.csv'
trades_path = dir_path + '/train_files/trades.csv'
options_path = dir_path + '/train_files/options.csv'

In [4]:
# Read the stock prices into a Dataframe.
stock_data = pd.read_csv(stock_prices_path)
trades_data = pd.read_csv(trades_path)
options_data = pd.read_csv(options_path)

/var/folders/49/57p15qz10bdb3cdxt3mldxgw0000gn/T/ipykernel_1882/1582623417.py:4: DtypeWarning: Columns (7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  options_data = pd.read_csv(options_path)


In [5]:
# Inspect the Dataframe
stock_data.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


#### Data Definitions 
    Goal: gain an understanding of the data features to inform the next step of the project.
    - Column names 
    - Data types 
    - Description of the columns 
    - Counts and percents unique values 

In [6]:
# Get the column names
stock_col_names = stock_data.columns
print(stock_col_names)

Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       'Target'],
      dtype='object')


In [7]:
# Inspect the data types
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332531 entries, 0 to 2332530
Data columns (total 12 columns):
 #   Column            Dtype  
---  ------            -----  
 0   RowId             object 
 1   Date              object 
 2   SecuritiesCode    int64  
 3   Open              float64
 4   High              float64
 5   Low               float64
 6   Close             float64
 7   Volume            int64  
 8   AdjustmentFactor  float64
 9   ExpectedDividend  float64
 10  SupervisionFlag   bool   
 11  Target            float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 198.0+ MB


In [16]:
# Description of the columns 

    # RowId appears to represent the date the data was collected and the ID code for the security
    # Date is of type object. We will change this to a datetime object
    # SecuritiesCode is an int that represents a stock that can be found in the stock_list file
    # Open - High - Low - Close represent the price of the security at those respective points
    # Volume is total amount of securities exchanged during this trading day.
    # Adjustment factor - Used if the stock is going to split/reverse split. 
    # ExpectedDividend - The value of the dividend if there is one. 
    # SupervisionFlag - a boolean value for whether or not the stock will be dropped

In [8]:
# Let's check for null values 
print(stock_data.isnull().sum())

RowId                     0
Date                      0
SecuritiesCode            0
Open                   7608
High                   7608
Low                    7608
Close                  7608
Volume                    0
AdjustmentFactor          0
ExpectedDividend    2313666
SupervisionFlag           0
Target                  238
dtype: int64


In [9]:
# Based on the null values, we can see that the expected dividend column is mostly null. Whether or not the stock recieves a dividend can be
# helpful in some trading strategies, but when we are looking at market as a whole, with this many null values, it makes sense to drop the column
# entirely. In addition, we can drop RowId since it is merely the Date and the securities code. 

stock_data = stock_data.drop(columns=['RowId','ExpectedDividend'])

In [12]:
# Let's check out the uniqueness of AdjustmentFactor and SupervisionFlag columns. I suspect that neither of these are a major driver of the market
# as a whole. Depending on the weight of a particular stock on the market, it the supervision flag might be relevant, but in practice, businesses 
# are generally performing poorly for a long time before they receive such a flag. 

stock_data['AdjustmentFactor'].unique()

# The adjustment factor is likely unhelpful for us here, so, we can drop the column.

array([ 1.        ,  0.5       ,  5.        ,  0.33333333,  0.83333333,
        0.25      ,  0.90909091,  0.1       , 10.        ,  0.2       ,
        0.95238095,  2.        ,  0.14285714,  0.66666667,  0.86956522,
        0.76923077,  0.8       ,  4.        , 20.        ])

In [14]:
stock_data['SupervisionFlag'].value_counts()
# Here we can see a large majority of the market does not have a supervision flag. We can drop this column as well.

False    2332001
True         530
Name: SupervisionFlag, dtype: int64

In [15]:
stock_data = stock_data.drop(columns=['SupervisionFlag', 'AdjustmentFactor'])

In [16]:
stock_data.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,Target
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,0.000730
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,0.012324
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,0.006154
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,0.011053
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,0.003026
